In [ ]:
# =============================================================================
# Cell 1: Clone Repository SAJA
# =============================================================================
!git clone https://github.com/DepthAnything/Depth-Anything-V2.git 2>/dev/null || echo "Repo exists"
print("✓ Repository cloned")

In [ ]:
# =============================================================================
# Cell 2: Patch untuk NumPy 2.x
# =============================================================================
# Patch transform.py
transform_file = '/kaggle/working/Depth-Anything-V2/depth_anything_v2/util/transform.py'
with open(transform_file, 'r') as f:
    content = f.read()
content = content.replace(
    'sample["image"] = cv2.resize(sample["image"], (width, height), interpolation=self.__image_interpolation_method)',
    'sample["image"] = cv2.resize(sample["image"], (int(width), int(height)), interpolation=self.__image_interpolation_method)'
)
with open(transform_file, 'w') as f:
    f.write(content)

# Patch dpt.py
dpt_file = '/kaggle/working/Depth-Anything-V2/depth_anything_v2/dpt.py'
with open(dpt_file, 'r') as f:
    content = f.read()
content = content.replace(
    'depth = F.interpolate(depth[:, None], (h, w), mode="bilinear", align_corners=True)[0, 0]',
    'depth = F.interpolate(depth[:, None], (int(h), int(w)), mode="bilinear", align_corners=True)[0, 0]'
)
with open(dpt_file, 'w') as f:
    f.write(content)

print("✅ Patched for NumPy 2.x")

In [ ]:
# =============================================================================
# Cell 3: Import Libraries
# =============================================================================
import os, sys, time, shutil, cv2, torch
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

sys.path.append('/kaggle/working/Depth-Anything-V2')
from depth_anything_v2.dpt import DepthAnythingV2

print(f"✓ NumPy: {np.__version__}")
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA: {torch.cuda.is_available()}")

In [ ]:
# =============================================================================
# Cell 4: Set Paths
# =============================================================================
IS_KAGGLE = os.path.exists('/kaggle/input')
BASE_PATH = Path('/kaggle/working')
RGB_DATASET_PATH = Path('/kaggle/input/ffb-localization-dataset/ffb_localization')
SYNTHETIC_DEPTH_RAW = BASE_PATH / 'datasets' / 'depth_synthetic_da2_raw'
YOLO_DATASET_PATH = BASE_PATH / 'datasets' / 'ffb_localization_depth_synthetic'

print(f"RGB dataset exists: {RGB_DATASET_PATH.exists()}")

In [ ]:
# =============================================================================
# Cell 5: Download Model
# =============================================================================
!mkdir -p /kaggle/working/checkpoints
!wget -q -O /kaggle/working/checkpoints/depth_anything_v2_vitl.pth \
    https://huggingface.co/depth-anything/Depth-Anything-V2-Large/resolve/main/depth_anything_v2_vitl.pth
print("✅ Model downloaded")

In [ ]:
# =============================================================================
# Cell 6: Load Model
# =============================================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DepthAnythingV2(encoder="vitl", features=256, out_channels=[256, 512, 1024, 1024])
model.load_state_dict(torch.load("/kaggle/working/checkpoints/depth_anything_v2_vitl.pth", 
                                  map_location=device, weights_only=True))
model.to(device).eval()
print(f"✅ Model loaded on {device}")

In [ ]:
# =============================================================================
# Cell 7: Scan Images
# =============================================================================
def scan_rgb_images(base_path):
    images = {}
    for split in ['train', 'val', 'test']:
        p = base_path / 'images' / split
        images[split] = sorted(list(p.glob('*.png')) + list(p.glob('*.jpg'))) if p.exists() else []
        print(f"{split}: {len(images[split])}")
    return images

rgb_images = scan_rgb_images(RGB_DATASET_PATH)

In [ ]:
# =============================================================================
# Cell 8: Test Single Image
# =============================================================================
raw_img = cv2.imread(str(rgb_images['train'][0]))
depth = model.infer_image(raw_img)
print(f"✅ Test OK! Depth shape: {depth.shape}, range: [{depth.min():.1f}, {depth.max():.1f}]")

In [ ]:
# =============================================================================
# Cell 9: Generate All Depth Maps
# =============================================================================
def save_depth_16bit(depth_np, path):
    d_min, d_max = depth_np.min(), depth_np.max()
    norm = (depth_np - d_min) / (d_max - d_min) if d_max > d_min else np.zeros_like(depth_np)
    path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(path), (norm * 65535).astype(np.uint16))

stats = {'success': 0, 'failed': 0}
start = time.time()

for split in ['train', 'val', 'test']:
    imgs = rgb_images.get(split, [])
    if not imgs: continue
    
    print(f"\n=== {split.upper()} ({len(imgs)} images) ===")
    
    # Create dirs
    (YOLO_DATASET_PATH / 'images' / split).mkdir(parents=True, exist_ok=True)
    (YOLO_DATASET_PATH / 'labels' / split).mkdir(parents=True, exist_ok=True)
    (SYNTHETIC_DEPTH_RAW / split).mkdir(parents=True, exist_ok=True)
    
    # Copy labels
    lbl_src = RGB_DATASET_PATH / 'labels' / split
    if lbl_src.exists():
        for f in lbl_src.glob('*.txt'):
            shutil.copy2(f, YOLO_DATASET_PATH / 'labels' / split / f.name)
    
    # Generate depth
    for p in tqdm(imgs, desc=split):
        try:
            img = cv2.imread(str(p))
            depth = model.infer_image(img)
            save_depth_16bit(depth, SYNTHETIC_DEPTH_RAW / split / p.name)
            save_depth_16bit(depth, YOLO_DATASET_PATH / 'images' / split / p.name)
            stats['success'] += 1
        except Exception as e:
            stats['failed'] += 1

print(f"\n✅ Done: {stats['success']} success, {stats['failed']} failed")
print(f"⏱️ Time: {(time.time()-start)/60:.1f} min")

In [ ]:
# =============================================================================
# Cell 10: Create YAML
# =============================================================================
from datetime import datetime
yaml_content = f"""path: {YOLO_DATASET_PATH.as_posix()}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['fresh_fruit_bunch']
"""
(YOLO_DATASET_PATH / 'dataset.yaml').write_text(yaml_content)
print("✅ YAML saved")

In [ ]:
# =============================================================================
# Cell 11: Create Archives
# =============================================================================
shutil.make_archive('/kaggle/working/ffb_synthetic_depth_yolo', 'zip', YOLO_DATASET_PATH)
shutil.make_archive('/kaggle/working/ffb_synthetic_depth_raw', 'zip', SYNTHETIC_DEPTH_RAW)
print(f"✅ ffb_synthetic_depth_yolo.zip: {os.path.getsize('/kaggle/working/ffb_synthetic_depth_yolo.zip')/1024/1024:.1f} MB")